# Method: VCT regression

In [3]:
#import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
import numpy as np
import yaml

from vessel_manoeuvring_models.visualization.plot import track_plot, plot
from phd.visualization.plot_ship import track_plots
from vessel_manoeuvring_models.prime_system import PrimeSystem
from phd.visualization.plot_prediction import plot_total_force, plot_force_components, predict, plot_compare_model_forces, plot_parameter_contributions
from phd.visualization.bokeh_plotting import create_tab
from vessel_manoeuvring_models.symbols import *
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify, remove_functions, prime
from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]
import statsmodels.api as sm
import sympy as sp
from sympy import Eq, symbols, latex
import paper
from paper import save_fig, save_eq
import vessel_manoeuvring_models
from phd.pipelines.regression_VCT.nodes import df_VCT_to_prime, regress_VCT
from phd.pipelines.regression_VCT.regression_pipeline import pipeline_with_rudder

In [40]:
#%reload_kedro
ship='wPCC'
ship_data = catalog.load(f"{ship}.ship_data")
model_loaders = catalog.load(f"{ship}.models_VCT")
models_rudder_VCT_loaders = catalog.load(f"{ship}.models_rudder_VCT")
df_VCT = catalog.load(f"{ship}.df_VCT_scaled")
exclude_parameters = catalog.load(f"params:{ship}.VCT_exclude_parameters")

[01/10/24 13:13:24] INFO     Loading data from 'wPCC.ship_data' (YAMLDataSet)...                ]8;id=495122;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=529048;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'wPCC.models_VCT' (PartitionedDataset)...        ]8;id=226315;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=956921;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'wPCC.models_rudder_VCT' (PartitionedDataset)... ]8;id=34214;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=429768;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'wPCC.df_VCT_scaled' (CSVDataSet)...             ]8;id=709146;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=174594;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:wPCC.VCT_exclude_parameters'             ]8;id=898930;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=490231;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\
                             (MemoryDataset)...                                                                    

## Mathematical rudder model

In [5]:
model_abkowitz = models_rudder_VCT_loaders['Abkowitz']()

In [18]:
model_abkowitz.show_subsystems()

hull: PrimeEquationSubSystem
propellers: PropellersSimpleSystem
rudders: AbkowitzRudderSystem
rudder_hull_interaction: RudderHullInteractionDummySystem
wind_force: DummyWindForceSystem


In [8]:
df_VCT_prime = df_VCT_to_prime(model=model_abkowitz, df_VCT=df_VCT)

In [10]:
regression_pipeline = pipeline_with_rudder(df_VCT_prime=df_VCT_prime, model=model_abkowitz)

In [12]:
regression_pipeline.keys()

dict_keys(['Rudder fx', 'Rudder fy', 'Rudder mz', 'resistance', 'resistance fy', 'resistance mz', 'Drift angle fx', 'Drift angle fy', 'Drift angle mz', 'Circle fx', 'Circle fy', 'Circle mz', 'Circle + Drift fx', 'Circle + Drift fy', 'Circle + Drift mz'])

In [13]:
regression = regression_pipeline['Rudder fy']

In [16]:
regression['eq']

Eq(Y_D, Y_{0} + Y_{deltadeltadelta}*delta**3 + Y_{delta}*delta)

In [17]:
for name,regression in regression_pipeline.items():
    display(regression['eq'])

Eq(X_R, X_{deltadelta}*delta**2)

Eq(Y_D, Y_{0} + Y_{deltadeltadelta}*delta**3 + Y_{delta}*delta)

Eq(N_D, N_{0} + N_{deltadeltadelta}*delta**3 + N_{delta}*delta - X_{thrustport}*thrust_port*y_p_port - X_{thruststbd}*thrust_stbd*y_p_stbd)

Eq(X_D, X_{0} + X_{deltadelta}*delta**2 + X_{thrustport}*thrust_port + X_{thruststbd}*thrust_stbd + X_{u}*u)

Eq(Y_D, Y_{0} + Y_{deltadeltadelta}*delta**3 + Y_{delta}*delta)

...

Eq(Y_D, Y_{0} + Y_{rrr}*r**3 + Y_{r}*r)

Eq(N_D, N_{0} + N_{rrr}*r**3 + N_{r}*r - X_{thrustport}*thrust_port*y_p_port - X_{thruststbd}*thrust_stbd*y_p_stbd)

Eq(X_D, X_{0} + X_{rr}*r**2 + X_{thrustport}*thrust_port + X_{thruststbd}*thrust_stbd + X_{u}*u + X_{vr}*r*v + X_{vv}*v**2)

Eq(Y_D, Y_{0} + Y_{rrr}*r**3 + Y_{r}*r + Y_{vrr}*r**2*v + Y_{vvr}*r*v**2 + Y_{vvv}*v**3 + Y_{v}*v)

Eq(N_D, N_{0} + N_{rrr}*r**3 + N_{r}*r + N_{vrr}*r**2*v + N_{vvr}*r*v**2 + N_{vvv}*v**3 + N_{v}*v - X_{thrustport}*thrust_port*y_p_port - X_{thruststbd}*thrust_stbd*y_p_stbd)

In [19]:
df_VCT['test type'].unique()

array(['Circle', 'Circle + Drift', 'Circle + rudder angle', 'Drift angle',
       'Rudder and drift angle', 'Rudder angle', 'Thrust variation',
       'self propulsion', 'Rudder angle resistance (no propeller)'],
      dtype=object)

In [27]:
df_VCT.groupby(by='test type').max()[['u','v','r','delta','thrust']]

,u,v,r,delta,thrust
test type,,,,,
Circle,0.961770,-0.000000,-0.023974,-0.000000,10.394889
Circle + Drift,0.961770,0.167010,0.047949,-0.000000,9.709370
Circle + rudder angle,0.961770,-0.000000,-0.023974,0.069813,9.291126
Drift angle,0.961184,-0.033565,0.000000,-0.000000,10.664554
Rudder and drift angle,0.959427,-0.067090,0.000000,0.069813,9.410107
Rudder angle,0.961770,-0.000000,0.000000,-0.052360,9.229330
Rudder angle resistance (no propeller),0.961770,-0.000000,0.000000,-0.087266,0.000000
Thrust variation,0.961770,-0.000000,0.000000,-0.174533,13.721516
self propulsion,0.961770,-0.000000,0.000000,-0.000000,9.130484


In [28]:
df_VCT.groupby(by='test type').min()[['u','v','r','delta','thrust']]

,u,v,r,delta,thrust
test type,,,,,
Circle,0.961770,-0.000000,-0.127864,-0.000000,9.249258
Circle + Drift,0.947079,-0.167010,-0.047949,-0.000000,0.000000
Circle + rudder angle,0.961770,-0.000000,-0.023974,-0.069813,9.228015
Drift angle,0.903768,-0.328945,0.000000,-0.000000,9.124859
Rudder and drift angle,0.959427,-0.067090,0.000000,-0.069813,9.381886
Rudder angle,0.961770,-0.000000,0.000000,-0.261799,9.095363
Rudder angle resistance (no propeller),0.961770,-0.000000,0.000000,-0.261799,0.000000
Thrust variation,0.480885,-0.000000,0.000000,-0.174533,1.360484
self propulsion,0.480885,-0.000000,0.000000,-0.000000,2.577043


In [30]:
eq_N_D = model_abkowitz.expand_subsystemequations(model_abkowitz.N_D_eq)

In [36]:
eq_N_D.subs([
    (v,0),
    (r,0),
    (delta,0),
    (thrust_port,0),
    (thrust_stbd,0),
])

Eq(N_D, N_{0})

In [41]:
exclude_parameters

{'Y0': 0,
 'N0': 0,
 'Yvdeltadelta': 0,
 'Yvvdelta': 0,
 'Yrdeltadelta': 0,
 'Yrrdelta': 0,
 'Yvrdelta': 0,
 'Nrrdelta': 0,
 'Nvrdelta': 0,
 'Nvdeltadelta': 0,
 'Nrdeltadelta': 0,
 'Nvvdelta': 0,
 'Xthrustport': 0.883,
 'Xthruststbd': 0.883}

In [54]:
model_,models = regress_VCT(model=model_abkowitz, df_VCT=df_VCT, pipeline=pipeline_with_rudder, exclude_parameters=exclude_parameters)

[01/10/24 13:25:22] INFO     Fitting:Rudder fx                                           ]8;id=217956;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py\regression_pipeline.py]8;;\:]8;id=824547;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py#297\297]8;;\

                    INFO     Fitting:Rudder fy                                           ]8;id=126798;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py\regression_pipeline.py]8;;\:]8;id=963719;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py#297\297]8;;\

                    INFO     Fitting:Rudder mz                                           ]8;id=443675;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py\regression_pipeline.py]8;;\:]8;id=665013;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py#297\297]8;;\

                    INFO     Fitting:resistance                                          ]8;id=630286;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py\regression_pipeline.py]8;;\:]8;id=383949;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py#297\297]8;;\

                    INFO     Fitting:resistance fy                                       ]8;id=200260;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py\regression_pipeline.py]8;;\:]8;id=908439;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py#297\297]8;;\

...

                    INFO                       Results: Ordinary least squares                         ]8;id=518096;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py\nodes.py]8;;\:]8;id=752524;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py#551\551]8;;\
                             ===================================================================                   
                             Model:              OLS              Adj. R-squared:     -0.000                       
                             Dependent Variable: X_D              AIC:                -2243.2433                   
                             Date:               2024-01-10 13:25 BIC:                -2237.6353                   
                             No. Observations:   122              Log-Likelihood:     1123.6                       
                             Df Model:           1                F-statistic:        0.9637                       
                             Df Residuals:       120              Prob (F-statistic): 0.328                        
                             R-squared:          0.008            Scale:              5.9565e-10                   
                             ---------------------------------------------------------------------                 
                                          Coef.    Std.Err.      t      P>|t|     [0.025    0.975]                 
                             ---------------------------------------------------------------------                 
                             Xvr         -0.0002     0.0002   -0.9817   0.3282   -0.0007    0.0002                 
                             const       -0.0000     0.0000   -4.4618   0.0000   -0.0000   -0.0000                 
                             -------------------------------------------------------------------                   
                             Omnibus:               9.106         Durbin-Watson:           1.792                   
                             Prob(Omnibus):         0.011         Jarque-Bera (JB):        9.022                   
                             Skew:                  -0.645        Prob(JB):                0.011                   
                             Kurtosis:              3.336         Condition No.:           102                     
                             ===================================================================                   
                                                                                                                   

                    INFO     Regression:Circle + Drift fy                                              ]8;id=770166;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py\nodes.py]8;;\:]8;id=48042;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py#549\549]8;;\

                    INFO                             Results: Ordinary least squares                   ]8;id=336057;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py\nodes.py]8;;\:]8;id=945282;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py#551\551]8;;\
                             =========================================================================             
                             =======                                                                               
                             Model:                  OLS              Adj. R-squared (uncentered):                 
                             0.347                                                                                 
                             Dependent Variable:     Y_D              AIC:                                         
                             -1926.0282                                                                            
                             Date:                   2024-01-10 13:25 BIC:                                         
                             -1920.4202                                                                            
                             No. Observations:       122              Log-Likelihood:                              
                             965.01                                                                                
                             Df Model:               2                F-statistic:                                 
                             33.37                                                                                 
                             Df Residuals:           120              Prob (F-statistic):                          
                             2.99e-12                                                                              
                             R-squared (uncentered): 0.357            Scale:                                       
                             8.0206e-09                                                                            
                             -------------------------------------------------------------------------             
                             ------------                                                                          
                                           Coef.       Std.Err.         t         P>|t|        [0.025              
                             0.975]                                                                                
                             -------------------------------------------------------------------------             
                             ------------                                                                          
                             Yvrr         -0.0531        0.0071      -7.4978      0.0000      -0.0671              
                             -0.0391                                                                               
                             Yvvr         -0.0774        0.0096      -8.1045      0.0000      -0.0963              
                             -0.0585                                                                               
                             -------------------------------------------------------------------------             
                             -------                                                                               
                             Omnibus:                  409.913            Durbin-Watson:                           
                             1.625                                                                                 
                             Prob(Omnibus):            0.000              Jarque-Bera (JB):                        
                             11.443                                                                       

                    INFO     Regression:Circle + Drift mz                                              ]8;id=985624;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py\nodes.py]8;;\:]8;id=892793;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py#549\549]8;;\

                    INFO                             Results: Ordinary least squares                   ]8;id=812095;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py\nodes.py]8;;\:]8;id=544113;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py#551\551]8;;\
                             =========================================================================             
                             =======                                                                               
                             Model:                  OLS              Adj. R-squared (uncentered):                 
                             0.902                                                                                 
                             Dependent Variable:     N_D              AIC:                                         
                             -2314.7936                                                                            
                             Date:                   2024-01-10 13:25 BIC:                                         
                             -2309.1855                                                                            
                             No. Observations:       122              Log-Likelihood:                              
                             1159.4                                                                                
                             Df Model:               2                F-statistic:                                 
                             565.4                                                                                 
                             Df Residuals:           120              Prob (F-statistic):                          
                             8.30e-62                                                                              
                             R-squared (uncentered): 0.904            Scale:                                       
                             3.3135e-10                                                                            
                             -------------------------------------------------------------------------             
                             ------------                                                                          
                                          Coef.       Std.Err.         t          P>|t|        [0.025              
                             0.975]                                                                                
                             -------------------------------------------------------------------------             
                             ------------                                                                          
                             Nvvr        -0.0432        0.0019      -22.2720      0.0000      -0.0471              
                             -0.0394                                                                               
                             Nvrr        -0.0091        0.0014       -6.3197      0.0000      -0.0119              
                             -0.0062                                                                               
                             -------------------------------------------------------------------------             
                             -------                                                                               
                             Omnibus:                  21.041            Durbin-Watson:                            
                             2.356                                                                                 
                             Prob(Omnibus):            0.000             Jarque-Bera (JB):                         
                             124.574                                                                      

In [57]:
regression = models['Circle + Drift mz']

In [49]:
list(regression.params.keys())

['Ydeltadeltadelta', 'Ydelta']

In [50]:
parameters = pd.Series()
for name, regression in models.items():
    parameters[name] = list(regression.params.keys())

In [51]:
parameters

Rudder fx                  [Xdeltadelta, const]
Rudder fy            [Ydeltadeltadelta, Ydelta]
Rudder mz            [Ndeltadeltadelta, Ndelta]
resistance                             [Xu, X0]
Drift angle fx                     [Xvv, const]
Drift angle fy                       [Yv, Yvvv]
Drift angle mz                       [Nvvv, Nv]
Circle fx                          [Xrr, const]
Circle fy                            [Yrrr, Yr]
Circle mz                            [Nr, Nrrr]
Circle + Drift fx                  [Xvr, const]
Circle + Drift fy                  [Yvrr, Yvvr]
Circle + Drift mz                  [Nvvr, Nvrr]
dtype: object

In [58]:
regression.eq

Eq(N_D, N_{0} + N_{rrr}*r**3 + N_{r}*r + N_{vrr}*r**2*v + N_{vvr}*r*v**2 + N_{vvv}*v**3 + N_{v}*v - X_{thrustport}*thrust_port*y_p_port - X_{thruststbd}*thrust_stbd*y_p_stbd)

In [94]:
eq_latex = sp.latex(regression.eq.lhs)
eq_latex_prime = f"{{{eq_latex}}}'"
eq_latex_prime

"{N_{D}}'"

In [143]:
def to_latex_prime(expression):
    eq_latex = sp.latex(expression)
    eq_latex_prime = f"${{{eq_latex}}}'$"
    eq_latex_prime = eq_latex_prime.replace('delta',r'\delta')
    return eq_latex_prime

In [144]:
name_replace={
    "Rudder":"Rudder angle",
    "resistance":"Self propulsion",
}

In [145]:
table_regression_pipeline = pd.DataFrame()
found = set([u,v,r,delta,thrust_port,thrust_stbd,p.Xthruststbd,p.Xthrustport,y_p_port, y_p_stbd,p.Y0,p.N0])
for name, regression in models.items():
    new_parameters = regression.eq.rhs.free_symbols - found
    found=found.union(regression.eq.rhs.free_symbols)
    
    test_type = str(name)
    test_type = test_type.replace(" fx","")
    test_type = test_type.replace(" fy","")
    test_type = test_type.replace(" mz","")
    table_regression_pipeline.loc[name,'Test type'] = name_replace.get(test_type,test_type) 
    table_regression_pipeline.loc[name,'Label'] = to_latex_prime(regression.eq.lhs)
    table_regression_pipeline.loc[name,'Features'] = " ".join([to_latex_prime(parameter) for parameter in new_parameters])

In [146]:
table_regression_pipeline

,Test type,Label,Features
Rudder fx,Rudder angle,${X_{R}}'$,${X_{\delta\delta}}'$
Rudder fy,Rudder angle,${Y_{D}}'$,${Y_{\delta}}'$ ${Y_{\delta\delta\delta}}'$
Rudder mz,Rudder angle,${N_{D}}'$,${N_{\delta}}'$ ${N_{\delta\delta\delta}}'$
resistance,Self propulsion,${X_{D}}'$,${X_{u}}'$ ${X_{0}}'$
Drift angle fx,Drift angle,${X_{D}}'$,${X_{vv}}'$
Drift angle fy,Drift angle,${Y_{D}}'$,${Y_{vvv}}'$ ${Y_{v}}'$
Drift angle mz,Drift angle,${N_{D}}'$,${N_{vvv}}'$ ${N_{v}}'$
Circle fx,Circle,${X_{D}}'$,${X_{rr}}'$
Circle fy,Circle,${Y_{D}}'$,${Y_{r}}'$ ${Y_{rrr}}'$
Circle mz,Circle,${N_{D}}'$,${N_{rrr}}'$ ${N_{r}}'$


In [147]:
table_regression_pipeline.to_csv(paper.file_path_with_nb_ref("regression_pipeline_abkowitz.csv", directory='tables'), index=False)

## Semi-empirical model

In [148]:
model = model_loaders['semiempirical_covered']()

In [149]:
model

{'semiempirical_covered': <bound method AbstractDataset.load of <phd.extras.datasets.model_dataset.ModularVesselSimulatorDataSet object at 0x7fc2eca92c80>>}